# Assignment 2A 
### Joost Schutte (12736023) & Dennis Lenferink (13397788)
<br>

# Introduction
Below, we first import all the necessary models and functions from the scikitlearn library.

In [1]:
import pandas as pd 
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

Next, we import the dataset. When printing the datatypes of the columns in the dataset below, we can see that there are 18 columns, most of which are integers or floats. However, not all of the datatypes are correct, as Weekend and Revenue should actually be integers, while OperatingSystems, Browser, Region and TrafficType should be strings so these categorical variables can be made into dummy variables.

In [2]:
df = pd.read_csv('data.csv')

print("The datatypes of the columns in the dataset are: \n")
print(df.dtypes)

The datatypes of the columns in the dataset are: 

Administrative               int64
Administrative_Duration    float64
Informational                int64
Informational_Duration     float64
ProductRelated               int64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                       object
OperatingSystems             int64
Browser                      int64
Region                       int64
TrafficType                  int64
VisitorType                 object
Weekend                       bool
Revenue                       bool
dtype: object


As mentioned, we want to change the Weekend and Revenue columns to integers, which we do below. Furthermore, we want to get dummies for the VisitorType, OperatingSystems, Browser, Region and TrafficType. To do so for the last four of these, we first have to convert the integers in to strings so the get_dummies function in sklearn recognizes that these columns should also be converted into dummies. Lastly, we drop the Month column, as instructed in the assignment. The final columns and their corresponding datatypes can be found below.

In [3]:
df["Weekend"] = df["Weekend"].astype(int)
df["Revenue"] = df["Revenue"].astype(int)

df["OperatingSystems"] = df["OperatingSystems"].astype(str)
df["Browser"] = df["Browser"].astype(str)
df["Region"] = df["Region"].astype(str)
df["TrafficType"] = df["TrafficType"].astype(str)

df = df.drop(columns="Month")

df = pd.get_dummies(df)

print(df.dtypes)

Administrative                     int64
Administrative_Duration          float64
Informational                      int64
Informational_Duration           float64
ProductRelated                     int64
                                  ...   
TrafficType_8                      uint8
TrafficType_9                      uint8
VisitorType_New_Visitor            uint8
VisitorType_Other                  uint8
VisitorType_Returning_Visitor      uint8
Length: 65, dtype: object


Below, we perform a train_test_split on the dataset, by first creating a variable that stores all predictor variables, and then performing the split using this new variable.

In [4]:
predictors = df.drop("Revenue", 1)

X_train, X_test, y_train, y_test = train_test_split(predictors, df["Revenue"], random_state=0)

print(f"X_train shape: {X_train.shape}") 
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (9247, 64)
y_train shape: (9247,)
X_test shape: (3083, 64)
y_test shape: (3083,)


# LinearSVC models
In the first three code cells below, we train a LinearSVC model on the dataset three times. For the first version of this model we use no scaling, for the second version we use the StandardScaler, and for the last version we use the MinMaxScaler.

For each of these models we use a pipeline to make sure the dataset is scaled correctly, after which the model is fit to the dataset. After creating the pipeline, we create a gridsearch in order to be able to find the optimal parameter that should be used in the model to optimize the test set score.

One last note that is important for both the LinearSVC and LogisticRegression models, is that we used dual=False in all versions of these two models. We use this option to speed up the model and to bypass problems with convergence. This setting is possible because the scikitlearn documentation states that dual can be set to false when the number of samples is larger than the number of features, which is the case for this dataset.

## LinearSVC without scaling
As can be seen in the output below, the LinearSVC model without scaling performs already pretty good. The model has a training set score of 0.885 and a test set score of 0.864. There is not a large gap between these two scores, which indicates no over- or underfitting. 

The best_params_ method of the gridsearch indicates that the best value for parameter C is 4.344, which, as mentioned before, results in a test set score of 0.864. To find this value for the best parameter we, for both the LinearSVC and the LogisticRegression models, ran the gridsearch multiple times while constantly narrowing down the range between which the gridsearch should look. The original range we used for all models except the KNeighborsClassifier model was 1,5. While performing the gridsearch and narrowing down the range, we made sure that the best value of the parameter was neither the smallest, nor the largest value we checked using the gridsearch.

In [5]:
lin_no_pipe = make_pipeline(LinearSVC(dual=False))

lin_no_param_grid = {"linearsvc__C":np.arange(4.3, 4.4, 0.001)}
lin_no_grid = GridSearchCV(lin_no_pipe, lin_no_param_grid, cv=3, n_jobs=-1)
lin_no_grid.fit(X_train, y_train)

print(f"Training set score: {lin_no_grid.score(X_train, y_train)}")
print(f"Test set score: {lin_no_grid.score(X_test, y_test)}")
print(f"Best parameters: {lin_no_grid.best_params_}")

Training set score: 0.8854763707148264
Test set score: 0.8640934155043789
Best parameters: {'linearsvc__C': 4.3440000000000145}


## LinearSVC with StandardScaler
The second version of LinearSVC, this time using the StandardScaler, has similar training and test set scores as the model without scaling; 0.886 and 0.864 respectively. Again, the closeness of these scores give no indication for over- or underfitting.

In this case however, it is clear that the best value for parameter C is very different, given that it is 0.010. Just like the previous model, gridsearch was used to find the optimal value, constantly narrowing down the range in which gridsearch should look for the best value.

In [6]:
lin_ss_pipe = make_pipeline(StandardScaler(), LinearSVC(dual=False))

lin_ss_param_grid = {"linearsvc__C":np.arange(0.001, 0.02, 0.001)}
lin_ss_grid = GridSearchCV(lin_ss_pipe, lin_ss_param_grid, cv=3, n_jobs=-1)
lin_ss_grid.fit(X_train, y_train)

print(f"Training set score: {lin_ss_grid.score(X_train, y_train)}")
print(f"Test set score: {lin_ss_grid.score(X_test, y_test)}")
print(f"Best parameter: {lin_ss_grid.best_params_}")

Training set score: 0.8860170866226884
Test set score: 0.8637690561141745
Best parameter: {'linearsvc__C': 0.010000000000000002}


## LinearSVC with MinMaxScaler
Once again, this time for the LinearSVC model using MinMaxScaler, the values of the training and test set scores are very close to the scores of the previous models. The training set score is 0.886 again, while the test set score is 0.864.

Just like before, however, the best value of parameter C is different for this model when compared to the other two models. The optimal value of C that we found using gridsearch is 4.000 for the LinearSVC model with MinMaxScaler.

As we found out using the gridsearch, the influence of C on the test set score is noticable, because the test set score goes down if C is set to an non-optimal value. It is interesting to note that the the three variations of the same model have such a different optimal value for which the test set score is the highest.

In [7]:
lin_mm_pipe = make_pipeline(MinMaxScaler(), LinearSVC(dual=False))

lin_mm_param_grid = {"linearsvc__C":np.arange(3.8, 4.2, 0.01)}
lin_mm_grid = GridSearchCV(lin_mm_pipe, lin_mm_param_grid, cv=3, n_jobs=-1)
lin_mm_grid.fit(X_train, y_train)

print(f"Training set score: {lin_mm_grid.score(X_train, y_train)}")
print(f"Test set score: {lin_mm_grid.score(X_test, y_test)}")
print(f"Best parameter: {lin_mm_grid.best_params_}")

Training set score: 0.8861252298042609
Test set score: 0.8640934155043789
Best parameter: {'linearsvc__C': 3.9999999999999956}


## LinearSVC model summary
As can be seen in the code blocks above, the LinearSVC model has pretty good performance overall. For all versions of the model with different scalers, the test set score is 0.864, which means that the model can make the correct prediction 86.4% of all cases. None of the scalers perform better than the others, which is pretty interesting.

# LogisticRegression models
In the following three code cells we do the same thing as we did with the LinearSVC models above. We train the model on the dataset three times, all with different scaling methods. The first method is no scaling, the second method is StandardScaler, and the last method is MinMaxScaler.

We again created pipelines to make sure the scaling and fitting of the model happens in the right order. Then, we use gridsearch to find the optimal value for the parameter of the LogisticRegression model; C. 

## LogisticRegression without scaling
For the LogisticRegression model without scaling, the training and test set scores can be found in the output below, as well as the best value for the parameter C. As can be seen, the training and test set scores (0.888 and 0.868 respectively) are, just like the LinearSVC models, relatively close to each other, meaning that there is probably no over- or underfitting. 

Just like we did for the LinearSVC models, we used a narrower and narrower range in a gridsearch to find the optimal value for the parameter of the LogisticRegression model. Here, this value is 4.410.

In [8]:
log_no_pipe = make_pipeline(LogisticRegression(dual=False, max_iter=1e6))

log_no_param_grid = {"logisticregression__C":np.arange(4.3, 4.5, 0.01)}
log_no_grid = GridSearchCV(log_no_pipe, log_no_param_grid, cv=3, n_jobs=-1)
log_no_grid.fit(X_train, y_train)

print(f"Training set score: {log_no_grid.score(X_train, y_train)}")
print(f"Test set score: {log_no_grid.score(X_test, y_test)}")
print(f"Best parameter: {log_no_grid.best_params_}")

Training set score: 0.8883962366172813
Test set score: 0.8676613687966267
Best parameter: {'logisticregression__C': 4.4099999999999975}


## LogisticRegression with StandardScaler
As can be seen below, the LogisticRegression model with a StandardScaler performs slightly better on the training set (with a score of 0.891) than the same model without scaling. The test set score (0.869) however, is the slightest bit lower than the model without scaling.

Interesting to notice is that the best value for parameter C is very different for the model with StandardScaler, when compared to the unscaled model. This value is 0.910.

In [9]:
log_ss_pipe = make_pipeline(StandardScaler(), LogisticRegression(dual=False))

log_ss_param_grid = {"logisticregression__C":np.arange(0.1, 1, 0.01)}
log_ss_grid = GridSearchCV(log_ss_pipe, log_ss_param_grid, cv=3, n_jobs=-1)
log_ss_grid.fit(X_train, y_train)

print(f"Training set score: {log_ss_grid.score(X_train, y_train)}")
print(f"Test set score: {log_ss_grid.score(X_test, y_test)}")
print(f"Best parameter: {log_ss_grid.best_params_}")

Training set score: 0.8905591002487293
Test set score: 0.8692831657476484
Best parameter: {'logisticregression__C': 0.9099999999999996}


## LogisticRegression with MinMaxScaler
The last version of the LogisticRegression model, with the MinMaxScaler, also performs similarly as the previous two models, with training and test set scores of 0.887 and 0.866 respectively. 

The best value for parameter C is comparable to the value in the model without scaling, at 4.770. As a reminder, this value was achieved by running a gridsearch on the model multiple times, each time narrowing the range in which the gridsearch has to look for the best value.

As was mentioned for the LinearSVC model, the value of parameter C has an influence on the test set score; being that the test set score decreases when the value of C is not optimalized. The best value for C results in a higher test set score, while a different arbitrary value for C results in a lower score.

In [10]:
log_mm_pipe = make_pipeline(MinMaxScaler(), LogisticRegression(dual=False, max_iter=1e6))

log_mm_param_grid = {"logisticregression__C":np.arange(4.7, 4.9, 0.01)}
log_mm_grid = GridSearchCV(log_mm_pipe, log_mm_param_grid, cv=3, n_jobs=-1)
log_mm_grid.fit(X_train, y_train)

print(f"Training set score: {log_mm_grid.score(X_train, y_train)}")
print(f"Test set score: {log_mm_grid.score(X_test, y_test)}")
print(f"Best parameter: {log_mm_grid.best_params_}")

Training set score: 0.8866659457121229
Test set score: 0.8663639312358092
Best parameter: {'logisticregression__C': 4.769999999999999}


## LogisticRegression model summary
Just like for the LinearSVC model, the LogisticRegression model performs pretty good, with test set scores of 0.868, 0.869 and 0.866 for models without scaling, StandardScaler and MinMaxScaler respectively. Based on just this metric, we can conclude that the model with StandardScaler performs slightly better than the other models. However, this might not be the case when looking at different metrics.

# KNeighborsClassifier models
The last type of model we used in this assignment is the KNeighborsClassifier model. Once again, we created three versions of this model; one without scaling, one with StandardScaler, and one with MinMaxScaler. 

Just like for all previous models, we created a pipeline for each version of this model to make sure that all operations were performed in the correct order. Lastly, a gridsearch was used to find the optimal value of the parameter n_neighbors.

## KNeighborsClassifier without scaling
As seen below, the first version of the KNeighborsClassifier has training and test set scores of 0.887 and 0.854 respectively. Again, these numbers give no indication of over- or underfitting because of the fact that they are relatively close to each other. 

The gridsearch we performed on the model shows that a value of 6 is optimal for the n_neighbors parameter to get a test set score that is as high as possible. 

In [11]:
"""KNEIGBORSCLASSIFIER, NO SCALING"""
kn_no_pipe = make_pipeline(KNeighborsClassifier())

kn_no_param_grid = {"kneighborsclassifier__n_neighbors":np.arange(1, 30, 1)}
kn_no_grid = GridSearchCV(kn_no_pipe, kn_no_param_grid, cv=3, n_jobs=-1)
kn_no_grid.fit(X_train, y_train)

print(f"Training set score: {kn_no_grid.score(X_train, y_train)}")
print(f"Test set score: {kn_no_grid.score(X_test, y_test)}")
print(f"Best parameter: {kn_no_grid.best_params_}")

Training set score: 0.8872066616199848
Test set score: 0.8543626337982485
Best parameter: {'kneighborsclassifier__n_neighbors': 6}


## KNeighborsClassifier with StandardScaler
The KNeighborsClassifier model with StandardScaler performs marginally worse than the model without any scaling. The training set score is 0.878, and the test set score is 0.848.

The optimal value for parameter n_neighbors is a bit higher than the optimal value in the model without scaling, but this has no significant meaning because a different scaling method is used.

In [12]:
"""KNEIGBORSCLASSIFIER, STANDARDSCALER"""
kn_ss_pipe = make_pipeline(StandardScaler(), KNeighborsClassifier())

kn_ss_param_grid = {"kneighborsclassifier__n_neighbors":np.arange(1, 30, 1)}
kn_ss_grid = GridSearchCV(kn_ss_pipe, kn_ss_param_grid, cv=3, n_jobs=-1)
kn_ss_grid.fit(X_train, y_train)

print(f"Training set score: {kn_ss_grid.score(X_train, y_train)}")
print(f"Test set score: {kn_ss_grid.score(X_test, y_test)}")
print(f"Best parameter: {kn_ss_grid.best_params_}")

Training set score: 0.8777982048231859
Test set score: 0.8481998053843659
Best parameter: {'kneighborsclassifier__n_neighbors': 9}


## KNeighborsClassifier with MinMaxScaler
The last version of the KNeighborsClassifier model has the MinMaxScaler applied to it. This results in a training set score of 0.859, and a test set score of 0.836, a little bit worse than both other versions of the same model.

For this version of the KNeighborsClassifier model, the best value for parameter n_neighbors is significantly higher than the value in the previous two versions of the model, with a value of 19. This value was once again achieved using a gridsearch with a range of 1 to 30 n_neighbors.

For this model the same applies as the LinearSVC and LogisticRegression models when it comes to the value of the model parameter, n_neighbors. When the value of the parameter is optimized using gridsearch, the test set score is higher than when this parameter value is arbitrarily chosen. 

In [13]:
"""KNEIGBORSCLASSIFIER, MINMAXSCALER"""
kn_mm_pipe = make_pipeline(MinMaxScaler(), KNeighborsClassifier())

kn_mm_param_grid = {"kneighborsclassifier__n_neighbors":np.arange(1, 30, 1)}
kn_mm_grid = GridSearchCV(kn_mm_pipe, kn_mm_param_grid, cv=3, n_jobs=-1)
kn_mm_grid.fit(X_train, y_train)

print(f"Training set score: {kn_mm_grid.score(X_train, y_train)}")
print(f"Test set score: {kn_mm_grid.score(X_test, y_test)}")
print(f"Best parameter: {kn_mm_grid.best_params_}")

Training set score: 0.8590894344111604
Test set score: 0.8358741485566007
Best parameter: {'kneighborsclassifier__n_neighbors': 19}


## KNeighborsClassifier model summary
Based on the test set scores of the KNeighborsClassifier models (0.854, 0.848 and 0.836 for the different scalers), we can see that this model is slightly less well suited for this dataset. However these scores are still not very bad. Furthermore, based on the test set scores, we can conclude that the model without any scaling actually performs the best out of the three models.

# Evaluation metrics
In this section a number of evaluation metrics such as precision, recall, f1-score, accuracy, macro average, and weighted average will be discussed. These metrics were obtained using the classification_report function from the sklearn library. 

For each of the three models that were used in this assignment, the metrics will be explained. In this explanation all three versions of each model will be discussed and compared, to find out what version of the model works best based on the different metrics.

## Evaluation metrics for the LinearSVC models
As can be seen in the three code blocks below, the evaluation metrics of all three versions of the LinearSVC model are very close to each other. This is somewhat surprising, but on the other hand not so much, given that the accuracy scores of these versions were very close as well. 

We can see that precision ranges between 0.74 and 0.75 for the LinearSVC models, while recall ranges from 0.30 to 0.31. The F1-score is the same for all three versions, with a score of 0.43. The macro and weighted averages are the same for all three models as well, with scores of 0.81, 0.64 and 0.68 for macro average, and 0.85, 0.86 and 0.84 for the weighted average. Lastly, when rounded, all three versions of the LinearSVC model have the same score for the ROC AUC-score: 0.64. 

The higher precision than recall in these models mean that we have fewer false positives, but we might have more false negatives. The F1-score, being a combination of these two, gives a more nuances view, with the macro and weighted averages doing even more so. The AUC-score for these models is not great, but it is by no means bad either.

In [14]:
lin_no_y_pred = lin_no_grid.predict(X_test)

print(classification_report(y_test, lin_no_y_pred))

print(f"ROC Area Under the Curve score: {roc_auc_score(y_test, lin_no_y_pred)}")

              precision    recall  f1-score   support

           0       0.87      0.98      0.92      2559
           1       0.74      0.31      0.43       524

    accuracy                           0.86      3083
   macro avg       0.81      0.64      0.68      3083
weighted avg       0.85      0.86      0.84      3083

ROC Area Under the Curve score: 0.6419253704184303


In [15]:
lin_ss_y_pred = lin_ss_grid.predict(X_test)

print(classification_report(y_test, lin_ss_y_pred))

print(f"ROC Area Under the Curve score: {roc_auc_score(y_test, lin_ss_y_pred)}")

              precision    recall  f1-score   support

           0       0.87      0.98      0.92      2559
           1       0.75      0.30      0.43       524

    accuracy                           0.86      3083
   macro avg       0.81      0.64      0.68      3083
weighted avg       0.85      0.86      0.84      3083

ROC Area Under the Curve score: 0.6402123622956247


In [16]:
lin_mm_y_pred = lin_mm_grid.predict(X_test)

print(classification_report(y_test, lin_mm_y_pred))

print(f"ROC Area Under the Curve score: {roc_auc_score(y_test, lin_mm_y_pred)}")

              precision    recall  f1-score   support

           0       0.87      0.98      0.92      2559
           1       0.75      0.30      0.43       524

    accuracy                           0.86      3083
   macro avg       0.81      0.64      0.68      3083
weighted avg       0.85      0.86      0.84      3083

ROC Area Under the Curve score: 0.6404077511193841


## Evaluation metrics for the LogisticRegression models
Just like for the LinearSVC models, the three versions of the LogisticRegression models have very similar scores on the various metrics, with very little difference between model versions.

We see that precision ranges from 0.72 to 0.75, while recall ranges from 0.32 to 0.36. The F1-score this time is not the same for all three versions, ranging from 0.45 to 0.48. The macro and weighted averages are also not the same for all versions of the model, but they are again very close to each other. The exact scores can be found in the outputs below. The ROC AUC-scores range from 0.64 to 0.66.

In this model, again, precision is higher than recall, meaning the models will have fewer false positives, but a higher number of false negatives. The AUC-score is comparable for all three versions of the model, and is also very close to all versions of the LinearSVC model.

In [17]:
log_no_y_pred = log_no_grid.predict(X_test)

print(classification_report(y_test, log_no_y_pred))

print(f"ROC Area Under the Curve score: {roc_auc_score(y_test, log_no_y_pred)}")

              precision    recall  f1-score   support

           0       0.88      0.97      0.92      2559
           1       0.72      0.36      0.48       524

    accuracy                           0.87      3083
   macro avg       0.80      0.66      0.70      3083
weighted avg       0.85      0.87      0.85      3083

ROC Area Under the Curve score: 0.6645625080169077


In [18]:
log_ss_y_pred = log_ss_grid.predict(X_test)

print(classification_report(y_test, log_ss_y_pred))

print(f"ROC Area Under the Curve score: {roc_auc_score(y_test, log_ss_y_pred)}")

              precision    recall  f1-score   support

           0       0.88      0.97      0.93      2559
           1       0.74      0.36      0.48       524

    accuracy                           0.87      3083
   macro avg       0.81      0.67      0.70      3083
weighted avg       0.86      0.87      0.85      3083

ROC Area Under the Curve score: 0.6662982617852274


In [19]:
log_mm_y_pred = log_mm_grid.predict(X_test)

print(classification_report(y_test, log_mm_y_pred))

print(f"ROC Area Under the Curve score: {roc_auc_score(y_test, log_mm_y_pred)}")

              precision    recall  f1-score   support

           0       0.88      0.98      0.92      2559
           1       0.75      0.32      0.45       524

    accuracy                           0.87      3083
   macro avg       0.81      0.65      0.69      3083
weighted avg       0.85      0.87      0.84      3083

ROC Area Under the Curve score: 0.6493635693809305


## Evaluation metrics for the KNeighborsClassifier models
For the last model in this assignment, what we see in the code blocks below is a bit different than what we saw for the other models. Here, the ranges of all metrics are a lot larger than in the previous models, which also means that a certain version of the model performs way better than the other versions.

As we can see, precision still has a marginal range, differing from 0.71 to 0.73. Recall and F1-score however, are a different story, with recall ranging from 0.06 to 0.23 and F1-score ranging from 0.11 to 0.34. The same can be said for the macro and weighted averages, the scores of which can be seen below, with larger ranges for the different metrics, however these are not as extreme as the other metrics. Lastly, the ROC AUC-score also differs more than it did in the previous models, ranging from 0.53 to 0.60.

Again, we can see that precision is higher than recall for these models. Furthermore, we see that the scores for the KNeighborsClassifier model without any scaling performs a lot better than the scaled versions of the model. This version performs better than the other versions for all of the performance metrics we used. However, still performs worse than the best verions of the previous two models.

In [20]:
kn_no_y_pred = kn_no_grid.predict(X_test)

print(classification_report(y_test, kn_no_y_pred))

print(f"ROC Area Under the Curve score: {roc_auc_score(y_test, kn_no_y_pred)}")

              precision    recall  f1-score   support

           0       0.86      0.98      0.92      2559
           1       0.73      0.23      0.34       524

    accuracy                           0.85      3083
   macro avg       0.80      0.60      0.63      3083
weighted avg       0.84      0.85      0.82      3083

ROC Area Under the Curve score: 0.6041937004256791


In [21]:
kn_ss_y_pred = kn_ss_grid.predict(X_test)

print(classification_report(y_test, kn_ss_y_pred))

print(f"ROC Area Under the Curve score: {roc_auc_score(y_test, kn_ss_y_pred)}")

              precision    recall  f1-score   support

           0       0.85      0.99      0.92      2559
           1       0.71      0.18      0.29       524

    accuracy                           0.85      3083
   macro avg       0.78      0.58      0.60      3083
weighted avg       0.83      0.85      0.81      3083

ROC Area Under the Curve score: 0.5822698811856969


In [22]:
kn_mm_y_pred = kn_mm_grid.predict(X_test)

print(classification_report(y_test, kn_mm_y_pred))

print(f"ROC Area Under the Curve score: {roc_auc_score(y_test, kn_mm_y_pred)}")

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      2559
           1       0.71      0.06      0.11       524

    accuracy                           0.84      3083
   macro avg       0.78      0.53      0.51      3083
weighted avg       0.82      0.84      0.77      3083

ROC Area Under the Curve score: 0.5262812883133619


# Final conclusion
After having taken a look at all versions of all models, we can see that there are small differences in performance between the versions and models. 

For the LinearSVC model, the best performing version is very hard to determine due to the close scores, but it probably is the unscaled version of the model, because the AUC-score is ever so slightly higher in this version.

For the LogisticRegression model, the best performing version was again a bit hard to determine due to the close score, but based on the slightly higher scores, the model with StandardScaler perfomed best out of the three versions of the model.

Lastly, for the KNeighborsClassifier model, the best performing versions was very easy to decide given that the scores were very different for all versions. The best performing version here is the one without any scaling.

Based on all scores we have seen here, and the best performing versions of all three models, we can say that the best performing model over all is the LogisticRegression model, with the highest scores on all evaluation metrics.